In [1]:
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import random
from PIL import Image
import shutil
import time
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, ResNet101
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from sklearn.preprocessing import LabelEncoder 

In [2]:
DATASET_PATH = "."

num_classes = 4 # ['Healthy', 'BrownSpot', 'Hispa', 'LeafBlast']

labels = ['Healthy', 'BrownSpot', 'Hispa', 'LeafBlast']

BATCH_SIZE = 50

EPOCHS = 100

In [3]:
def SaveImage (pImageObject, pSrcImagePath, pDstPath, pMethodName):
    
    head, tail = os.path.split(pSrcImagePath)
    filename, file_extension = os.path.splitext(tail)
    
    dst_path = pDstPath + "\\" + filename + "_" + pMethodName + file_extension
    pImageObject.save (dst_path, dpi = pImageObject.info['dpi'], quality = 95)

def FlipTB (pSrcImage, pDstPath):
    
    imageObject = Image.open(pSrcImage)
    hori_flippedImage = imageObject.transpose(Image.FLIP_TOP_BOTTOM)
    
    SaveImage (hori_flippedImage, pSrcImage, pDstPath, "FlipTB")

def FlipLR (pSrcImage, pDstPath):
    
    imageObject = Image.open(pSrcImage)
    hori_flippedImage = imageObject.transpose(Image.FLIP_LEFT_RIGHT)
    
    SaveImage (hori_flippedImage, pSrcImage, pDstPath, "FlipLR")

def Rotate (pSrcImage, pDstPath):
    
    imageObject = Image.open(pSrcImage)
    hori_flippedImage = imageObject.transpose(Image.ROTATE_90)
    
    SaveImage (hori_flippedImage, pSrcImage, pDstPath, "Rot90")

def Scale (pSrcImage, pDstPath):
    
    imageObject = Image.open(pSrcImage)
    scale = 1 + (random.randrange (10, 20) / 100)
    
    width = int (imageObject.size[0] * scale)
    height = int (imageObject.size[1] * scale)
    
    scaled_image = imageObject.resize((width, height), Image.LANCZOS)
    
    left = int ((scaled_image.size[0] - imageObject.size[0])/2)
    upper = int (((scaled_image.size[1] - imageObject.size[1])/2))
    right = scaled_image.size[0] - left
    lower = scaled_image.size[1] - upper
    
    cropped_image = scaled_image.crop ((left, upper, right, lower))
    SaveImage (cropped_image, pSrcImage, pDstPath, "SC")
    

def Translate (pSrcImage, pDstPath):
    pass

augment_funcs = [FlipTB, FlipLR, Rotate, Scale]

def AugmentImage (pSrcImageList, pDstPath, pAugmentMethod):
    
    with ThreadPoolExecutor () as executor:
        
        #running_tasks = []
        
        for image in pSrcImageList:
            
            task = executor.submit (augment_funcs[pAugmentMethod], image, pDstPath)
            #running_tasks.append (task)
            
        #for running_task in running_tasks:
            #running_task.result()

            
def CopyImageList (pSrcImageList, pDstPath):
    
    with ThreadPoolExecutor () as executor:
        
        for image in pSrcImageList:
            
            executor.submit (shutil.copy, image, pDstPath)

def AugmentClass (pSrcPath, pDstPath, pClassName): # pClassName can be Healthy, BrownSpot, Hispa, and LeafBlast
    
    src_path = pSrcPath + "\\" + pClassName
    files = [src_path + "\\" + f for f in listdir(src_path) if isfile(join(src_path, f))] # List of all images
    
    images_to_add = 1800 - len (files)
    
    num_methods = len(augment_funcs)
    num_images_to_augment = int (min (images_to_add / num_methods, len (files)))
    print (num_images_to_augment)                                
    
    with ThreadPoolExecutor () as executor:
        
        #running_tasks = []
        
        for i in range (num_methods):
        
            images_copy = files.copy()           
            
            images_to_augment = random.sample (images_copy, num_images_to_augment)
            
            executor.submit (AugmentImage, images_to_augment.copy(), pDstPath + "\\" + pClassName, i)
            executor.submit (CopyImageList, images_copy, pDstPath + "\\" + pClassName)


    
    

def AugmentData (pPath = DATASET_PATH):
    
    if (pPath):
        
        print ("Path Present")
        path = pPath
        
    else:
        
        print ("Path Not Present, using default")
        path = "."
    
    augmented_path = path
    path += "\\RiceLeafs\\train"
        
    if not (os.path.exists(path)):
        raise Exception ("Data not present!")
        
    augmented_path += "\\RiceLeafs\\train_augmented"
    
    if (os.path.exists(augmented_path)):
        
        print ("Augmented Data already present")
        return
    
    os.makedirs(augmented_path)
    print ("Created Dir: {0}:", augmented_path)
    
    with ThreadPoolExecutor() as executor:
        
        for label in labels:
            
            os.makedirs (augmented_path + "\\" + label)
            executor.submit (AugmentClass, path, augmented_path, label)
            
    
    

In [4]:
start_time = int (time.time () * 1000)
AugmentData()
end_time = int (time.time () * 1000)

print ("Time taken to Augment Data: {0} ms", end_time - start_time)

Path Present
Augmented Data already present
Time taken to Augment Data: {0} ms 1


In [5]:
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)

def GetFileListAndLabelEncoding (pPath):
    
    if not (os.path.exists(pPath)):
        raise Exception ("Data not present!")
        
    file_list = []
    class_list = []
    
    le = LabelEncoder ()
    le.fit (labels)
    
    for label in labels:
        
        print (label)
        path = pPath + "\\" + label
        count = 0 
        list_dir = listdir (path)
        for f in list_dir:
            
            print (str(count) + "/" + str (len (list_dir)))
            count += 1
            
            if (isfile(join(path, f))):
                
                file_list.append (img_to_array(load_img(join (path, f), target_size = IMG_DIM)))
                class_list.append (label)
                
    return np.array(file_list), le.transform(class_list)

def GetTrainAndValidationGenerator ():
    
    train_datagen = ImageDataGenerator(rescale = 1./255)
    val_datagen = ImageDataGenerator(rescale = 1./255)
    
    #train_images, train_labels_enc = GetFileListAndLabelEncoding (DATASET_PATH + "\\RiceLeafs\\train_augmented")
    #val_images, val_labels_enc = GetFileListAndLabelEncoding (DATASET_PATH + "\\RiceLeafs\\validation")
    
    train_generator = train_datagen.flow_from_directory (directory = DATASET_PATH + "\\RiceLeafs\\train_augmented", batch_size = BATCH_SIZE)
    val_generator = val_datagen.flow_from_directory (directory = DATASET_PATH + "\\RiceLeafs\\validation", batch_size = BATCH_SIZE)
    
    return train_generator, val_generator
    

In [6]:
def GetResNetModel (pModelName):

    if pModelName == 'ResNet50':
        base_model = ResNet50 (weights = 'imagenet', include_top = False, input_shape = (IMG_HEIGHT,IMG_WIDTH,3))
    elif pModelName == 'ResNet101':
        base_model = ResNet101 (weights = 'imagenet', include_top = False, input_shape = (IMG_HEIGHT,IMG_WIDTH,3))
    else:
        raise Exception ("Unknown ResNet model")
        
    split_at = int (0.9 * len (base_model.layers)) 
    
    for layer in base_model.layers[:split_at]:
        layer.trainable = False
                    
    for layer in base_model.layers[split_at:]:
        layer.trainable = True
        
    x = GlobalAveragePooling2D () (base_model.output)
    
    x = Dense (512, activation = 'relu') (x)
    
    x = Dense (num_classes, activation = 'softmax', name = 'FC_new') (x)
    
    model = Model (inputs = base_model.input, outputs = x)
    
    return model
    

In [7]:
model = GetResNetModel('ResNet101')

sgd = SGD (lr = 0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy',
              optimizer = sgd,
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

 conv2_block2_add (Add)         (None, 128, 128, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, 128, 128, 25  0           ['conv2_block2_add[0][0]']       
                                6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 128, 128, 64  16448       ['conv2_block2_out[0][0]']       
                                )                                                                 
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 128, 128, 64  256        ['conv2_block3_1_conv[0][0]']    
 ization) 

 conv3_block2_3_conv (Conv2D)   (None, 64, 64, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_3_bn (BatchNormal  (None, 64, 64, 512)  2048       ['conv3_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_add (Add)         (None, 64, 64, 512)  0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_bn[0][0]']      


C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


                                                                                                  
 conv3_block2_out (Activation)  (None, 64, 64, 512)  0           ['conv3_block2_add[0][0]']       
                                                                                                  
 conv3_block3_1_conv (Conv2D)   (None, 64, 64, 128)  65664       ['conv3_block2_out[0][0]']       
                                                                                                  
 conv3_block3_1_bn (BatchNormal  (None, 64, 64, 128)  512        ['conv3_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 64, 64, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
          

...

                                                                                                  
 conv4_block13_3_bn (BatchNorma  (None, 32, 32, 1024  4096       ['conv4_block13_3_conv[0][0]']   
 lization)                      )                                                                 
                                                                                                  
 conv4_block13_add (Add)        (None, 32, 32, 1024  0           ['conv4_block12_out[0][0]',      
                                )                                 'conv4_block13_3_bn[0][0]']     
                                                                                                  
 conv4_block13_out (Activation)  (None, 32, 32, 1024  0          ['conv4_block13_add[0][0]']      
                                )                                                                 
                                                                                                  
 conv4_blo

                                )                                                                 
                                                                                                  
 conv4_block16_3_bn (BatchNorma  (None, 32, 32, 1024  4096       ['conv4_block16_3_conv[0][0]']   
 lization)                      )                                                                 
                                                                                                  
 conv4_block16_add (Add)        (None, 32, 32, 1024  0           ['conv4_block15_out[0][0]',      
                                )                                 'conv4_block16_3_bn[0][0]']     
                                                                                                  
 conv4_block16_out (Activation)  (None, 32, 32, 1024  0          ['conv4_block16_add[0][0]']      
                                )                                                                 
          

 conv4_block19_3_conv (Conv2D)  (None, 32, 32, 1024  263168      ['conv4_block19_2_relu[0][0]']   
                                )                                                                 
                                                                                                  
 conv4_block19_3_bn (BatchNorma  (None, 32, 32, 1024  4096       ['conv4_block19_3_conv[0][0]']   
 lization)                      )                                                                 
                                                                                                  
 conv4_block19_add (Add)        (None, 32, 32, 1024  0           ['conv4_block18_out[0][0]',      
                                )                                 'conv4_block19_3_bn[0][0]']     
                                                                                                  
 conv4_block19_out (Activation)  (None, 32, 32, 1024  0          ['conv4_block19_add[0][0]']      
          

                                                                                                  
 conv4_block22_3_conv (Conv2D)  (None, 32, 32, 1024  263168      ['conv4_block22_2_relu[0][0]']   
                                )                                                                 
                                                                                                  
 conv4_block22_3_bn (BatchNorma  (None, 32, 32, 1024  4096       ['conv4_block22_3_conv[0][0]']   
 lization)                      )                                                                 
                                                                                                  
 conv4_block22_add (Add)        (None, 32, 32, 1024  0           ['conv4_block21_out[0][0]',      
                                )                                 'conv4_block22_3_bn[0][0]']     
                                                                                                  
 conv4_blo

 conv5_block2_2_conv (Conv2D)   (None, 16, 16, 512)  2359808     ['conv5_block2_1_relu[0][0]']    
                                                                                                  
 conv5_block2_2_bn (BatchNormal  (None, 16, 16, 512)  2048       ['conv5_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_2_relu (Activatio  (None, 16, 16, 512)  0          ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 16, 16, 2048  1050624     ['conv5_block2_2_relu[0][0]']    
                                )                                                                 
          

In [8]:
train_generator, val_generator = GetTrainAndValidationGenerator () 

history = model.fit(train_generator, steps_per_epoch = 50,  epochs = 100, validation_data = val_generator,  validation_steps = 50, verbose = 1)

Found 7196 images belonging to 4 classes.
Found 671 images belonging to 4 classes.
Epoch 1/100
50/50 [==============================] - 463s 8s/step - loss: 1.3010 - accuracy: 0.3684 - val_loss: 1.4311 - val_accuracy: 0.2012
Epoch 2/100
50/50 [==============================] - 274s 5s/step - loss: 1.2222 - accuracy: 0.4279
Epoch 3/100
50/50 [==============================] - 273s 5s/step - loss: 1.1883 - accuracy: 0.4547
Epoch 4/100
50/50 [==============================] - 254s 5s/step - loss: 1.1612 - accuracy: 0.4764
Epoch 5/100
50/50 [==============================] - 267s 5s/step - loss: 1.1549 - accuracy: 0.4840
Epoch 6/100
50/50 [==============================] - 253s 5s/step - loss: 1.1379 - accuracy: 0.4940
Epoch 7/100
50/50 [==============================] - 266s 5s/step - loss: 1.1005 - accuracy: 0.5208
Epoch 8/100
50/50 [==============================] - 254s 5s/step - loss: 1.1160 - accuracy: 0.4948
Epoch 9/100
50/50 [==============================] - 254s 5s/step - loss: 1

50/50 [==============================] - 93s 2s/step - loss: 0.4362 - accuracy: 0.8337
Epoch 78/100
50/50 [==============================] - 90s 2s/step - loss: 0.4551 - accuracy: 0.8284
Epoch 79/100
50/50 [==============================] - 99s 2s/step - loss: 0.3696 - accuracy: 0.8584
Epoch 80/100
50/50 [==============================] - 99s 2s/step - loss: 0.3927 - accuracy: 0.8532
Epoch 81/100
50/50 [==============================] - 87s 2s/step - loss: 0.3965 - accuracy: 0.8522
Epoch 82/100
50/50 [==============================] - 92s 2s/step - loss: 0.4005 - accuracy: 0.8482
Epoch 83/100
50/50 [==============================] - 95s 2s/step - loss: 0.3989 - accuracy: 0.8462
Epoch 84/100
50/50 [==============================] - 345s 7s/step - loss: 0.4157 - accuracy: 0.8400
Epoch 85/100
50/50 [==============================] - 374s 8s/step - loss: 0.3860 - accuracy: 0.8564
Epoch 86/100
50/50 [==============================] - 373s 8s/step - loss: 0.4027 - accuracy: 0.8428
Epoch 87/1

In [9]:
len (model.layers)

348